<a href="https://colab.research.google.com/github/kaisacht/Project/blob/main/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#khai báo các thư viện cần dùng
from __future__ import print_function 
from tensorflow import keras
import numpy as np
import pandas as pd

In [ ]:
#khởi tạo ngẫu nhiên nhưng với các giá trị cố định
np.random.seed(813306)

In [ ]:
#tải file lên
from google.colab import files
uploaded = files.upload()

In [ ]:
#chia nhãn ra khỏi bộ dữ liệu
def readucr(filename):
    data = np.loadtxt(filename, delimiter = ',')
    Y = data[:,0]
    X = data[:,1:]
    return X, Y

In [ ]:
#tính toán tập nhãn và tập dữ liệu tương ứng với taaoj train và test
x_train, y_train = readucr('dataset_3class_2day_TRAIN.txt')
x_test, y_test = readucr('dataset_3class_2day_TEST.txt')

In [ ]:
#in thử các giá trị của tập x
print(x_test,y_test)

In [ ]:
#thực hiện train 5000 lần
nb_epochs = 5000

In [ ]:
#tính lại tập train và tập test
x_train, y_train = readucr('dataset_3class_2day_TRAIN.txt')
x_test, y_test = readucr('dataset_3class_2day_TEST.txt')
#số lượng class
nb_classes =len(np.unique(y_test))
#đánh lại nhãn là từ 0 do bô jduwx liệu gắn nhãn từ 1
y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
#tính tham số batch
batch_size = min(x_train.shape[0]/10, 16)
#?
Y_train = keras.utils.to_categorical(y_train, nb_classes)
Y_test = keras.utils.to_categorical(y_test, nb_classes)
#tính giá trị trung bình của x   
x_train_mean = x_train.mean()
#tính phương sai của x
x_train_std = x_train.std()
#chuẩn hóa lại các giá trị của x
x_train = (x_train - x_train_mean)/(x_train_std)

# x_test_min = np.min(x_test, axis = 1, keepdims=1)
# x_test_max = np.max(x_test, axis = 1, keepdims=1)
x_test = (x_test - x_train_mean)/(x_train_std)
     
#x_train = x_train.reshape(x_train.shape + (1,))
#x_test = x_test.reshape(x_test.shape + (1,))
#xây dụng model
x = keras.layers.Input(x_train.shape[1:])
y= keras.layers.Dropout(0.1)(x)
y = keras.layers.Dense(500, activation='relu')(y)
y = keras.layers.Dropout(0.2)(y)
y = keras.layers.Dense(500, activation='relu')(y)
y = keras.layers.Dropout(0.2)(y)
y = keras.layers.Dense(500, activation = 'relu')(y)
y = keras.layers.Dropout(0.3)(y)
out = keras.layers.Dense(nb_classes, activation='softmax')(y)

model = keras.models.Model(inputs=x, outputs=out)
     
optimizer = keras.optimizers.Adadelta()    
model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
     
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = 'loss', factor=0.5,
                      patience=200, min_lr=0.1)
#tranning   
hist = model.fit(x_train, Y_train, batch_size=batch_size, epochs=nb_epochs,
              verbose=1, validation_data=(x_test, Y_test), 
                #callbacks = [TestCallback((x_train, Y_train)), reduce_lr, keras.callbacks.TensorBoard(log_dir='./log'+fname, histogram_freq=1)])
                 callbacks=[reduce_lr])
    
    #Print the testing results which has the lowest training loss.


In [ ]:
#in ra quá trình train
log = pd.DataFrame(hist.history)
print(log)

In [ ]:
#in ra lần train có giá trị loss nhỏ nhất
log_np = np.array(log)
log_np
min_index = (np.argmin(log_np, axis=0))
print(log_np[min_index[0]])